# Processing
Demonstration of Processing endpoints.

In [4]:
import utils.DemoClient as client
import jwt
import json
from owslib.csw import CatalogueServiceWeb

## Initialise

### Client
We instantiate a client to interact with the platform. The client dynamically registers with the Authorisation Server to take part in UMA (User Managed Access) flows through which authorization is obtained for scoped access resources on behalf of the user.

In [6]:
#-------------------------------------------------------------------------------
# Initialise client
#-------------------------------------------------------------------------------
base_domain = "demo.eoepca.org"
platform_domain = "test." + base_domain
base_url = "https://" + platform_domain
demo = client.DemoClient(base_url)
demo.register_client()
demo.save_state()

client_id: b5832c30-cb26-4d13-9803-4ebcb99d13e9
Client state saved to file: state.json


### User
User authenticates and the client receives an ID Token (JWT) that represents the user.

In [7]:
#-------------------------------------------------------------------------------
# Authenticate as UserA and get ID Token
#-------------------------------------------------------------------------------
USER_NAME="bob"
USER_PASSWORD="defaultPWD"
user_id_token = demo.get_id_token(USER_NAME, USER_PASSWORD)

[Request] GET => https://test.demo.eoepca.org/.well-known/uma2-configuration
<Response [200]>
token_endpoint: https://test.demo.eoepca.org/oxauth/restv1/token
[Request] POST => https://test.demo.eoepca.org/oxauth/restv1/token


## Workspace

In [ ]:
workspace_url = "https://workspace-api." + base_domain
workspace_access_token = None
workspace_prefix = "demo-user-"

### Workspace: Get Details
Retrieve access information to the user workspace S3 bucket (bucket name, access key and secret access key)

In [ ]:
# Workspace - Get Details
workspace_name =  workspace_prefix + USER_NAME.lower()
response, workspace_access_token = demo.workspace_get_details(workspace_url, workspace_name, id_token=user_id_token, access_token=workspace_access_token)
workspace_details = response.json()
print(json.dumps(workspace_details["storage"], indent = 2))

### Inspect User Workspace
Using the access credentials, the user can inspect his S3 bucket.

In [ ]:
# Bucket details
bucket_name = workspace_details["storage"]["credentials"]["bucketname"]
s3_access = workspace_details["storage"]["credentials"]["access"]
s3_secret = workspace_details["storage"]["credentials"]["secret"]

In [ ]:
# Quick hack S3 access
import boto3

# Init S3 session for Creodias
S3_ENDPOINT = "https://cf2.cloudferro.com:8080"
session = boto3.session.Session()
s3resource = session.resource('s3', aws_access_key_id=s3_access, aws_secret_access_key=s3_secret, endpoint_url=S3_ENDPOINT)

# List bucket contents
bucket = s3resource.Bucket(bucket_name)
for obj in bucket.objects.all():
    print(' ->', obj.key)

### Upload Application Package and register it
User uploads the application package to his workspace

In [ ]:
object = s3resource.Object(bucket_name, 'application-package/s-expression/s-expression-0_0_2.cwl')
result = object.put(Body=open('../data/s-expression-cwl.cwl', 'rb'))
res = result.get('ResponseMetadata')
if res.get('HTTPStatusCode') == 200:
    print('Application package uploaded successfully')
else:
    print('Application package not uploaded')

User registers the Application Package through the Resource Catalog API

In [ ]:
resource_url= f'{bucket_name}/application-package/s-expression/s-expression-0_0_2.cwl'
demo.workspace_register(workspace_url, workspace_name,resource_url, id_token=user_id_token, access_token=workspace_access_token)

User inspects the registered Application Package through the Resource Catalogue API

In [ ]:
headers = {
    'Authorization': 'Bearer ' + user_id_token
}
app_id= "s-expression"
workspace_endpoint = f'https://resource-catalogue.{workspace_prefix}{USER_NAME.lower()}.{base_domain}/csw'
print(f"{USER_NAME}'s workspace (catalogue): {workspace_endpoint}")
csw = CatalogueServiceWeb(workspace_endpoint,timeout=30,headers=headers)
csw.getrecords2(maxrecords=10)
print(f"{USER_NAME}'s workspace records:\n")
for rec in csw.records:
    print(f'identifier: {csw.records[rec].identifier}\ntype: {csw.records[rec].type}\ntitle: {csw.records[rec].title}\n')
print(f"URL for Application {app_id}:")
csw.records[app_id].references

## ADES
The ADES provides user-specific endpoints, using a URL path prefix.

In [8]:
# ADES URLs
ades_base_url = "http://ades." + base_domain
ades_wps_url = ades_base_url + "/" + USER_NAME + "/zoo"; print("ADES WPS endpoint:", ades_wps_url)
ades_proc_url = ades_base_url + "/" + USER_NAME + "/wps3"; print("ADES API Processes endpoint:", ades_proc_url)

ADES WPS endpoint: http://ades.demo.eoepca.org/bob/zoo
ADES API Processes endpoint: http://ades.demo.eoepca.org/bob/wps3


### ADES API Processes

In [9]:
ades_access_token = None
app_name = "s-expression-0_0_2"

#### ADES: List Processes
**GET {service_url}/processes**<br>
Provides a list of all processes 

In [10]:
# API Processes - List Processes
response, ades_access_token, process_ids = demo.proc_list_processes(ades_proc_url, id_token=user_id_token, access_token=ades_access_token)
print("Processes:", process_ids)

[Request] GET => http://ades.demo.eoepca.org/bob/wps3/processes
[Process List] = 200 (OK)
Processes: ['DeployProcess', 'GetStatus', 'UndeployProcess', 's-expression-0_0_2']


#### ADES: Deploy Application
**POST {service_url}/processes**<br>
Deploys application to the ADES.<br>
Request body is json in which the input is the Application Package describing the application.<br>
The Application Package is a CWL Workflow that is typically provided as an href with content type **_application/atom+xml_** or **_application/cwl_**. The url to the application package can be either a http or s3 endpoint. In this example we will use the application package previously uploaded to Bob's workspace.

In [ ]:
# API Processes - Deploy Application
#response, ades_access_token = demo.proc_deploy_application(ades_proc_url, "../data/app-deploy-body-atom.json", id_token=user_id_token, access_token=ades_access_token)

response, access_token = demo.proc_deploy_application(ades_proc_url, "../data/app-deploy-body-cwl-S3.json", id_token=user_id_token, access_token=ades_access_token)

Check that the application has been correctly deployed

In [ ]:
# Check deploy - list processes again
response, ades_access_token, process_ids = demo.proc_list_processes(ades_proc_url, id_token=user_id_token, access_token=ades_access_token)
print("Processes:", process_ids)

#### ADES: Get Application Details
**GET {service_url}/processes/{application_name}**<br>
Provides details of the deployed application<br>
The response includes the API Processes json application description.

In [ ]:
# API Processes - Get Application Details
response, ades_access_token = demo.proc_get_app_details(ades_proc_url, app_name, id_token=user_id_token, access_token=ades_access_token)
print(json.dumps(response.json(), indent = 2))

#### ADES: Execute Application

**POST {service_url}/processes/{application_name}/jobs**<br>
Request body is json as defined by API Processes to define the inputs and outputs, consistent with the CWL Workflow application package.<br>
The response returns **201 CREATED** to indicate that the job has been successfully initiated.<br>
The response **Location header** provides the path (/processes/{application_name}/jobs/{job_id}) to follow the job status.

In [ ]:
# API Processes - Execute Application
response, ades_access_token, job_location_path = demo.proc_execute_application(ades_proc_url, app_name, "../data/app-execute-body.json", id_token=user_id_token, access_token=ades_access_token)

#### ADES: Job Status

**GET {service_url}/watchjob/processes/{application_name}/jobs/{job_id}**<br>
Checks the status of the previously submitted job - using the URL returned in the Location header of the execute request.<br>
The response body json provides a status string (success/running/failed) and a % progress.<br>
In the case of a failure then a descriptive message is provided.

In [ ]:
# API Processes - Job Status
response, ades_access_token, status = demo.proc_get_job_status(ades_base_url, job_location_path, id_token=user_id_token, access_token=ades_access_token)
print(json.dumps(response.json(), indent = 2))

In [ ]:
# API Processes - Job Status (keep polling for completion)
response, ades_access_token, status = demo.proc_poll_job_completion(ades_base_url, job_location_path, interval=10, id_token=user_id_token, access_token=ades_access_token)

In [ ]:
print(response.status_code, response.headers, response.text)

#### ADES: Job Result
**GET {service_url}/watchjob/processes/{application_name}/jobs/{job_id}/result**<br>
Returns details of the outputs for a successful job execution.<br>
The response body provides json data that includes the reference to the STAC file that indexes the processing outputs.

In [ ]:
# API Processes - Job Result
response, ades_access_token, stacCatalogUri = demo.proc_get_job_result(ades_base_url, job_location_path, id_token=user_id_token, access_token=ades_access_token)
print(json.dumps(response.json(), indent = 2))

From the S3 url let's retrieve the folder in the bucket where the result files are stored.

In [ ]:
result = json.loads(response.json()["outputs"][0]["value"]["inlineValue"])["StacCatalogUri"]
print(f"Result Url: {result}")
result_folder_name = result.rsplit('/', 1)[0].rsplit(':', 1)[1].replace(bucket_name+"/","")
print(f"Result foldername: {result_folder_name}")

#### ADES: List Jobs

In [ ]:
# API Processes - List Jobs
response, ades_access_token, job_ids = demo.proc_list_jobs(ades_proc_url, app_name, id_token=user_id_token, access_token=ades_access_token)
print("Jobs:", job_ids)

#### ADES: Undeploy Application
**DELETE {service_url}/processes/{application_name}**<br>
Undeploys application from the ADES

In [ ]:
# API Processes - Undeploy Application
response, ades_access_token = demo.proc_undeploy_application(ades_proc_url, app_name, id_token=user_id_token, access_token=ades_access_token)

In [ ]:
# Check undeploy - list processes
response, ades_access_token, process_ids = demo.proc_list_processes(ades_proc_url, id_token=user_id_token, access_token=ades_access_token)
print("Processes:", process_ids)

#### Workspace: Inspect results
Let's inspect the result files in the user S3 bucket.

In [ ]:
# Quick hack S3 access
import boto3

# Init S3 session for Creodias
S3_ENDPOINT = "https://cf2.cloudferro.com:8080"
session = boto3.session.Session()
s3resource = session.resource('s3', aws_access_key_id=s3_access, aws_secret_access_key=s3_secret, endpoint_url=S3_ENDPOINT)

# List bucket contents
bucket = s3resource.Bucket(bucket_name)
for obj in bucket.objects.filter(Prefix=result_folder_name):
    print(' ->', obj.key)